##### Copyright 2019 The TensorFlow Authors.

In [11]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/programmers_guide/keras) and [eager execution](https://www.tensorflow.org/programmers_guide/eager). The following is sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [12]:
import os
import time

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [13]:
path_to_file = "data/divina_syll_textonly.txt"  # tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### Read the data

First, look in the text:

In [14]:
# Read, then decode for py2 compat.
text = open(path_to_file, "rb").read().decode(encoding="utf-8")
# length of text is the number of characters in it
print("Length of text: {} characters".format(len(text)))

Length of text: 873431 characters


In [15]:
# Take a look at the first 250 characters in text
print(text[:250])



 |Nel |mez|zo |del |cam|min |di |no|stra |vi|ta             
 |mi |ri|tro|vai |per |u|na |sel|va o|scu|ra,               
 |ché |la |di|rit|ta |via |e|ra |smar|ri|ta.                

 |Ahi |quan|to a |dir |qual |e|ra è |co|sa |du|ra           
 |e


In [16]:
# The unique characters in the file
vocab = sorted(set(text))
print("{} unique characters".format(len(vocab)))

80 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `preprocessing.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [17]:
example_texts = ["abcdefg", "xyzèòà+èàùò"]

chars = tf.strings.unicode_split(example_texts, input_encoding="UTF-8")
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z', b'\xc3\xa8', b'\xc3\xb2', b'\xc3\xa0', b'+', b'\xc3\xa8', b'\xc3\xa0', b'\xc3\xb9', b'\xc3\xb2']]>

Now create the `preprocessing.StringLookup` layer:

In [18]:
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab))

In [19]:
len(set(vocab))

80

In [20]:
ids_from_chars.vocab_size()

82

In [21]:
set(ids_from_chars.get_vocabulary()) - set(vocab)

{'', '[UNK]'}

It converts form tokens to character IDs, padding with `0`:

In [22]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[35, 36, 37, 38, 39, 40, 41], [56, 57, 58, 67, 72, 65, 1, 67, 65, 75, 72]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `preprocessing.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `preprocessing.StringLookup` layer so that the padding and `[UNK]` tokens are set the same way.

In [23]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, encoding="UTF-8"
)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [24]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z', b'\xc3\xa8', b'\xc3\xb2', b'\xc3\xa0', b'[UNK]', b'\xc3\xa8', b'\xc3\xa0', b'\xc3\xb9', b'\xc3\xb2']]>

In [25]:
tf.print(tf.strings.reduce_join(chars))

abcdefgxyzèòà[UNK]èàùò


You can `tf.strings.reduce_join` to join the characters back into strings. 

In [26]:
tf.print(tf.strings.reduce_join(chars, axis=-1))

["abcdefg" "xyzèòà[UNK]èàùò"]


In [27]:
vocab

['\n',
 ' ',
 '!',
 '"',
 '(',
 ')',
 ',',
 '-',
 '.',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'y',
 'z',
 '|',
 '«',
 '»',
 'È',
 'Ë',
 'Ï',
 'à',
 'ä',
 'è',
 'é',
 'ë',
 'ì',
 'ï',
 'ò',
 'ó',
 'ö',
 'ù',
 'ü',
 '—',
 '‘',
 '’',
 '“',
 '”']

In [28]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [29]:
def ids_from_text(text):
    return ids_from_chars(tf.strings.unicode_split(text, input_encoding="UTF-8"))

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [30]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, "UTF-8"))
all_ids

<tf.Tensor: shape=(873431,), dtype=int64, numpy=array([2, 2, 3, ..., 2, 2, 2])>

In [31]:
all_ids = ids_from_text(text)
all_ids

<tf.Tensor: shape=(873431,), dtype=int64, numpy=array([2, 2, 3, ..., 2, 2, 2])>

In [36]:
all_ids

<tf.Tensor: shape=(873431,), dtype=int64, numpy=array([2, 2, 3, ..., 2, 2, 2])>

In [37]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(100):
    print(chars_from_ids(ids).numpy().decode("utf-8"))

In [24]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [25]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
    print(text_from_ids(seq))

tf.Tensor(b'\n\n |Nel |mez|zo |del |cam|min |di |no|stra |vi|ta             \r\n |mi |ri|tro|vai |per |u|na |sel|va o', shape=(), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [26]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy().decode("UTF-8"))



 |Nel |mez|zo |del |cam|min |di |no|stra |vi|ta             
 |mi |ri|tro|vai |per |u|na |sel|va o
|scu|ra,               
 |ché |la |di|rit|ta |via |e|ra |smar|ri|ta.                

 |Ahi |quan|t
o a |dir |qual |e|ra è |co|sa |du|ra           
 |e|sta |sel|va |sel|vag|gia e |a|spra e |for|te    
        
 |che |nel |pen|sier |ri|no|va |la |pa|u|ra!                

 |Tan|t’ è |a|ma|ra |che |po
|co è |più |mor|te;             
 |ma |per |trat|tar |del |ben |ch’ i’ |vi |tro|vai,         
 |di|


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [27]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [28]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [29]:
dataset = sequences.map(split_input_target)

In [68]:
tf.print(input_example)

print("CIAO")

[52 4 60 ... 44 60 54]
CIAO


In [30]:
for input_example, target_example in dataset.take(10):
    print("Input:")
    tf.print(text_from_ids(input_example))
    print("Target:")
    tf.print(text_from_ids(target_example))

Input:


 |Nel |mez|zo |del |cam|min |di |no|stra |vi|ta             
 |mi |ri|tro|vai |per |u|na |sel|va 
Target:

 |Nel |mez|zo |del |cam|min |di |no|stra |vi|ta             
 |mi |ri|tro|vai |per |u|na |sel|va o
Input:
|scu|ra,               
 |ché |la |di|rit|ta |via |e|ra |smar|ri|ta.                

 |Ahi |quan|
Target:
scu|ra,               
 |ché |la |di|rit|ta |via |e|ra |smar|ri|ta.                

 |Ahi |quan|t
Input:
o a |dir |qual |e|ra è |co|sa |du|ra           
 |e|sta |sel|va |sel|vag|gia e |a|spra e |for|te   
Target:
 a |dir |qual |e|ra è |co|sa |du|ra           
 |e|sta |sel|va |sel|vag|gia e |a|spra e |for|te    
Input:
        
 |che |nel |pen|sier |ri|no|va |la |pa|u|ra!                

 |Tan|t’ è |a|ma|ra |che |p
Target:
       
 |che |nel |pen|sier |ri|no|va |la |pa|u|ra!                

 |Tan|t’ è |a|ma|ra |che |po
Input:
|co è |più |mor|te;             
 |ma |per |trat|tar |del |ben |ch’ i’ |vi |tro|vai,         
 |di
Target:
co è |più |mor|te;            

In [31]:
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [32]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [33]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : [b'|fia |te|sti|mo|nio al |ver |che |la |di|spen|sa.          \n\n |Tu |la|sce|rai |o|gne |co|sa |di|let|'
 b'|guer|ra;           \n |ma |or |si |fa |to|glien|do or |qui |or |qui|vi           \n |lo |pan |che \xe2\x80\x99l '
 b'  \r\n\n |Ma |dim|mi, |se |tu |sai, |per|ch\xc3\xa9 |tai |crol|li          \r\n |di\xc3\xa8 |dian|zi \xe2\x80\x99l |mon|te, e |per'
 b'al|tro o|stel|lo.     \n\n |Quin|ci ad|di|vien |ch\xe2\x80\x99 E|sa|\xc3\xb9 |si |di|par|te             \n |per |se|me |d'
 b'ra|t\xc3\xa8s |e |Ti|gri                \r\n |ve|der |mi |par|ve u|scir |d\xe2\x80\x99 u|na |fon|ta|na,            \r\n |e'
 b'\xe2\x80\x99 e|ra,        \n |per |gui|sa |d\xe2\x80\x99 o|riz|zon|te |che |ri|schia|ri.           \n\n |E |s\xc3\xac |co|me al |sa|'
 b'|ti al |col|lo, e |tro|ve|rai |la |so|ga            \r\n |che \xe2\x80\x99l |tien |le|ga|to, o |a|ni|ma |con|fu|s'
 b'r |co|s\xc3\xac |di|nan|zi ad |un |ch\xe2\x80\x99 al |pas|so          \r\n |pas|sa|va |Sti|ge |con |le |pian|te a|sciut|'

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([ 3, 44, 55,  3, 47, 61, 45, 59, 59,  2, 39, 55, 61,  3, 44, 55,  3,
       54, 55, 60,  3, 56, 61, 58, 56, 55, 59, 45,  3, 60, 55,  3, 48, 49,
       53, 13,  3, 41, 54, 44,  3, 41, 59,  3, 48, 41, 58, 44, 52, 65,  2,
       37, 49, 52, 52,  3, 48, 45,  3, 45, 54, 44, 61, 58, 45,  3, 65, 55,
       61, 58,  3, 59, 49, 47, 48, 60,  3, 41, 59,  3, 65, 45, 60,  8,  3,
       23,  3, 46, 45, 41, 58, 12,  2, 34, 48, 45, 54,  8,  3, 60])>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outpts one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [35]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [36]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(
            rnn_units, return_sequences=True, return_state=True
        )
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [75]:
model = MyModel2(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

In [74]:
class MyModel2(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru1 = tf.keras.layers.GRU(
            rnn_units, return_sequences=True, return_state=True
        )
        self.gru2 = tf.keras.layers.GRU(
            rnn_units, return_sequences=True, return_state=True
        )
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = [self.gru1.get_initial_state(x), self.gru2.get_initial_state(x)]

        x, state1 = self.gru1(x, initial_state=states[0], training=training)
        x, state2 = self.gru2(x, initial_state=states[1], training=training)

        states = [state1, state2]

        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [38]:
len(ids_from_chars.get_vocabulary())

83

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_training.png?raw=1)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details asee the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [76]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(
        example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)"
    )

(64, 100, 83) # (batch_size, sequence_length, vocab_size)


In [77]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 83), dtype=float32, numpy=
array([[ 0.00166124,  0.00178732,  0.00443886, ..., -0.0009829 ,
        -0.00363873, -0.00246529],
       [-0.00012814,  0.0033676 , -0.00018548, ..., -0.0052246 ,
        -0.00708352, -0.00788479],
       [-0.00659129,  0.0059875 , -0.00074197, ..., -0.00611988,
        -0.00578771, -0.00500283],
       ...,
       [-0.00086557,  0.00194827, -0.00041457, ..., -0.00952651,
        -0.00324467, -0.01349054],
       [-0.00258243,  0.00263016, -0.00483718, ..., -0.01201002,
        -0.00435651, -0.01685471],
       [-0.00273035,  0.00173474, -0.00408678, ..., -0.01488177,
        -0.00300407, -0.0123313 ]], dtype=float32)>

In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [42]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  21248     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  85075     
Total params: 4,044,627
Trainable params: 4,044,627
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [79]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
tf.print(text_from_ids(sampled_indices))

i”Qg?”!(là’(l“teSä“«|T
GuEVDhMË.dP
OöZuQà,;.x.!g Rq‘rüD.AivjpfshmN:


This gives us, at each timestep, a prediction of the next character index:

In [80]:
sampled_indices

array([44, 82, 29, 42, 14, 82,  5,  7, 46,  0, 66, 80,  7, 46, 81, 54, 40,
       31, 67, 81, 61, 60, 32,  2, 21, 55, 19, 34, 18, 43, 25, 64, 11, 39,
       28,  2, 71, 74, 19, 80, 50, 32, 37, 64, 25, 43, 24,  3, 76, 65, 11,
        0, 79, 67, 18, 24, 31, 21, 42,  9, 62, 56,  9, 52, 38, 35, 28, 77,
       18, 11, 15, 44, 56, 45, 50, 41, 53, 43, 47, 26, 12,  3, 27, 75, 35,
       55, 29, 66,  9, 13, 11, 57, 11,  5, 42,  4, 30, 51, 79, 52])

Decode these to see the text predicted by this untrained model:

In [81]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy().decode("utf-8"))
print()

print("Next Char Predictions:")
print(text_from_ids(sampled_indices).numpy().decode("utf8"))

Input:
  |com|pa|gna          
 |pic|cio|la |da |la |qual |non |fui |di|ser|to.            

 |L’ un |li|t

Next Char Predictions:
i”Qg?”!(là’(l“teSä“«|T
GuEVDhMË.dP
OöZuQà,;.x.!g Rq‘rüD.AivjpfshmN:


In [54]:
text_from_ids(input_example_batch[0]).numpy().decode("utf-8")

'il |trag|gi,  \n |per|ch’ e’ |co|ru|sca |sì |co|me |tu |ri|di;              \n\n |ma |non |so |chi |tu '

## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [82]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [83]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print(
    "Prediction shape: ",
    example_batch_predictions.shape,
    " # (batch_size, sequence_length, vocab_size)",
)
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 83)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.414427


In [84]:
target_example_batch

<tf.Tensor: shape=(64, 100), dtype=int64, numpy=
array([[60, 38, 49, ..., 60, 54, 49],
       [60, 53, 40, ..., 72, 60, 36],
       [38, 49, 46, ..., 42, 40, 48],
       ...,
       [51, 55, 36, ..., 38, 49, 53],
       [36, 60, 47, ...,  4,  4,  4],
       [39, 40, 46, ...,  4,  4,  4]])>

In [149]:
example_batch_predictions

<tf.Tensor: shape=(64, 100, 81), dtype=float32, numpy=
array([[[-1.77826313e-03,  1.48778781e-07,  1.55782944e-03, ...,
         -1.23497797e-04,  3.99828237e-03,  5.27239265e-03],
        [ 1.25536369e-03,  2.91570090e-04,  1.96321821e-03, ...,
         -6.96434116e-04,  7.11056683e-03,  3.43400356e-03],
        [ 3.48293153e-03,  7.54389912e-04,  1.37351081e-03, ...,
         -7.38847652e-04,  1.08625134e-02,  5.85250137e-03],
        ...,
        [-2.65783798e-02, -1.16850901e-02, -9.87099484e-03, ...,
          1.81808956e-02, -1.13374405e-02, -4.93007293e-03],
        [-2.68553253e-02, -1.21323867e-02, -1.07222833e-02, ...,
          1.87985599e-02, -1.25934789e-02, -4.97386279e-03],
        [-2.69706175e-02, -1.24254664e-02, -1.13606453e-02, ...,
          1.91299971e-02, -1.34498188e-02, -4.95460909e-03]],

       [[ 1.47263450e-03,  4.68797516e-04, -8.36860505e-04, ...,
          6.99016906e-04,  4.54668188e-03,  3.88545124e-03],
        [ 6.96595479e-03, -1.69393781e-03, -2.46

In [85]:
loss(target_example_batch, example_batch_predictions)

<tf.Tensor: shape=(), dtype=float32, numpy=4.414427>

A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [124]:
tf.exp(mean_loss).numpy()

80.53362

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [86]:
model.compile(optimizer="adam", loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [61]:
# Directory where the checkpoints will be saved
checkpoint_dir = "./training_checkpoints"
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, save_weights_only=True
)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [62]:
EPOCHS = 35

In [87]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/35
136/136 [==============================] - 20s 131ms/step - loss: 2.7697
Epoch 2/35
136/136 [==============================] - 19s 135ms/step - loss: 1.3682
Epoch 3/35
136/136 [==============================] - 20s 142ms/step - loss: 1.2162
Epoch 4/35
136/136 [==============================] - 20s 139ms/step - loss: 1.1237
Epoch 5/35
136/136 [==============================] - 19s 135ms/step - loss: 1.0533
Epoch 6/35
136/136 [==============================] - 19s 137ms/step - loss: 1.0021
Epoch 7/35
136/136 [==============================] - 20s 139ms/step - loss: 0.9631
Epoch 8/35
136/136 [==============================] - 20s 138ms/step - loss: 0.9305
Epoch 9/35
136/136 [==============================] - 19s 136ms/step - loss: 0.8998
Epoch 10/35
136/136 [==============================] - 19s 137ms/step - loss: 0.8694
Epoch 11/35
136/136 [==============================] - 19s 138ms/step - loss: 0.8424
Epoch 12/35
136/136 [==============================] - 19s 137ms/step - lo

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [64]:
a = np.array([1, 2, 3])

In [127]:
(a) / np.sum(np.exp(a))

array([0.0331204 , 0.06624079, 0.09936119])

In [126]:
(a / 20) / np.sum(np.exp(a / 20))

array([0.01506806, 0.03013613, 0.04520419])

In [88]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "" or "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(["", "[UNK]"])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float("inf")] * len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())],
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(
            inputs=input_ids, states=states, return_state=True
        )
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature
        # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [89]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [91]:
start = time.time()
states = None
next_char = tf.constant([" "])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result[0].numpy().decode("utf-8"), "\n\n" + "_" * 80)

print(f"\nRun time: {end - start}")

         
 |voi |co|no|sce|te, e |ciò, |ch’ i’ |pos|si |vi |ma|ni,       
 |a|ven|do |più |che ’l |mon|do |sì |gra|va,           
 |nel |mon|do |suo |quel |ch’ è |da |sé |sciol|ta».     

 |Al|lor |si |vol|se a |ri|guar|dar |lo |no|stro             
 |con |l’ ar|mo|nia |che |tem|pe|ri e|van|te,           
 |fan|no |do|lo|re, e |al |do|lor |fe|ne|stra.               

 |Non |cre|do |che |la |sca|let|ta |cir|ce, |lun|ga              
 |per |l’ al|ber |giu|sti|zï|on |che |noz|ze |seg|gio.            

 |Si|mi|le |moi |vi|sta; e |quel|la |di |co|lui              
 |che |fu|ron |di |sti|pit|to|ria |di|ser|ta,               

 |tal |o|ra a |me |ve|ni|te, ar|che |son |ri|spon|de,          
 |e |die|ce |pas|si |fem|mi|na |via |sag|gia».              

 |Lo |somem, |quan|do |dis|se:« |Fi|gliuol |mio,           
 |ché |per|der |vin|cer |che |le |vie |sì |fat|ti ar|te».       

 |Al|lor |mi |dis|se:« |Quel |da |cu’ |il |di|sio               
 |ciò |che ’n |grot|tian |den|tro |da |l 

____________

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [ ]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result, "\n\n" + "_" * 80)


print(f"\nRun time: {end - start}")

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [ ]:
tf.saved_model.save(one_step_model, "one_step")
one_step_reloaded = tf.saved_model.load("one_step")

In [ ]:
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [ ]:
class CustomTraining(MyModel):
    @tf.function
    def train_step(self, inputs):
        inputs, labels = inputs
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.loss(labels, predictions)
        grads = tape.gradient(loss, model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

        return {"loss": loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

In [ ]:
model.fit(dataset, epochs=1)

Or if you need more control, you can write your own complete custom training loop:

In [ ]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs["loss"])

        if batch_n % 50 == 0:
            template = "Epoch {} Batch {} Loss {}"
            print(template.format(epoch + 1, batch_n, logs["loss"]))

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print("Epoch {} Loss: {:.4f}".format(epoch + 1, mean.result().numpy()))
    print("Time taken for 1 epoch {} sec".format(time.time() - start))
    print("_" * 80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))